In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("day1").getOrCreate()

df1 = spark.read.csv('join1.csv', header= True, inferSchema= True )
df2 = spark.read.csv('join2.csv', header= True, inferSchema= True )

df1.join(df2, df1.txnno == df2.tno).select('custno').show()
print(" ========== left join results===========")
df1.join(df2, df1.txnno == df2.tno, 'left').drop('tno', 'txndate').show()
print(" ========== right join results===========")
df1.join(df2, df1.txnno == df2.tno, 'right').drop('txnno', 'txndate').show()
print(" ========== full join results===========")
df1.join(df2, df1.txnno == df2.tno, 'full').drop( 'txndate').show()

+-------+
| custno|
+-------+
|4007024|
|4006742|
|4002199|
+-------+

 ========== left join results===========
+-----+------+-------+-------+
|txnno|amount| custno|spendby|
+-----+------+-------+-------+
|    0| 40.33|4007024| credit|
|    1|198.44|4006742| credit|
|    2|  5.58|   NULL|   NULL|
|    3|198.19|4002199| credit|
|    4| 98.81|   NULL|   NULL|
+-----+------+-------+-------+

 ========== right join results===========
+------+---+-------+-------+
|amount|tno| custno|spendby|
+------+---+-------+-------+
| 40.33|  0|4007024| credit|
|198.44|  1|4006742| credit|
|198.19|  3|4002199| credit|
|  NULL|  7|6768689|   cash|
+------+---+-------+-------+

 ========== full join results===========
+-----+------+----+-------+-------+
|txnno|amount| tno| custno|spendby|
+-----+------+----+-------+-------+
|    0| 40.33|   0|4007024| credit|
|    1|198.44|   1|4006742| credit|
|    2|  5.58|NULL|   NULL|   NULL|
|    3|198.19|   3|4002199| credit|
|    4| 98.81|NULL|   NULL|   NULL|
| NU

In [4]:
from pyspark.sql.functions import *


print(" ========== left anti join results===========")
df1.join(df2, df1.txnno == df2.tno, 'left_anti').show()
print(" ========== left semi join results===========")
df1.join(df2, df1.txnno == df2.tno, 'left_semi').show()

df1.join(df2, df1.txnno == df2.tno, 'left').filter(col('tno').isNull() |
                                                   col('txnno').isNull()).show()

 ========== left anti join results===========
+-----+----------+------+
|txnno|   txndate|amount|
+-----+----------+------+
|    2|06-01-2011|  5.58|
|    4|12-17-2011| 98.81|
+-----+----------+------+

 ========== left semi join results===========
+-----+----------+------+
|txnno|   txndate|amount|
+-----+----------+------+
|    0|06-26-2011| 40.33|
|    1|05-26-2011|198.44|
|    3|06-05-2011|198.19|
+-----+----------+------+

+-----+----------+------+----+------+-------+
|txnno|   txndate|amount| tno|custno|spendby|
+-----+----------+------+----+------+-------+
|    2|06-01-2011|  5.58|NULL|  NULL|   NULL|
|    4|12-17-2011| 98.81|NULL|  NULL|   NULL|
+-----+----------+------+----+------+-------+



In [5]:
travel_df = spark.read.csv('travel.csv', header = True)
travel_df.show()

df = travel_df.groupBy("cust_id").agg( first('origin').alias('source'),
                                      last('destination').alias('destination'),
                                       sum('price').alias('total_cost'))
df.show()

+-------+---------+---------+-----------+-----+
|cust_id|flight_id|   origin|destination|price|
+-------+---------+---------+-----------+-----+
|      1|       f3|    kochi|  Mangalore| 1800|
|      1|       f1|    delhi|  hyderabad| 2500|
|      2|       f2|  Ayodhya|    chennai| 3000|
|      1|       f2|hyderabad|      kochi| 1700|
|      2|       f1|   Mumbai|    Ayodhya| 4000|
+-------+---------+---------+-----------+-----+

+-------+-------+-----------+----------+
|cust_id| source|destination|total_cost|
+-------+-------+-----------+----------+
|      1|  kochi|      kochi|    6000.0|
|      2|Ayodhya|    Ayodhya|    7000.0|
+-------+-------+-----------+----------+



In [6]:
from pyspark.sql.functions import *
# Sample data
data = [
    ("India", "SL", "India"),
    ("SL", "Aus", "Aus"),
    ("SA", "Eng", "Eng"),
    ("Eng", "NZ", "NZ"),
    ("Aus", "India", "India"),
]

columns = ["Team_1", "Team_2", "Winner"]
df = spark.createDataFrame(data, columns)
df.show()

+------+------+------+
|Team_1|Team_2|Winner|
+------+------+------+
| India|    SL| India|
|    SL|   Aus|   Aus|
|    SA|   Eng|   Eng|
|   Eng|    NZ|    NZ|
|   Aus| India| India|
+------+------+------+



In [7]:
df1= df.select("Team_1")
df2= df.select("Team_2")
union_df = df1.unionAll(df2) \
              .groupBy('Team_1') \
              .agg(count("Team_1").alias("matches_played")) \
              .withColumnRenamed("Team_1", 'team_name')
union_df.show()

join_df = union_df.join(df, union_df.team_name == df.Winner, 'left') \
                  .drop("Team_1", "Team_2")
join_df.show()

results_df = join_df.groupBy("team_name", "matches_played") \
                    .agg(count("Winner").alias("no_of_wins"))
results_df.show()

from pyspark.sql.types import *

final_df = results_df.withColumn("matches_played", col("matches_played").cast(IntegerType())) \
                      .withColumn("no_of_wins", col("no_of_wins").cast(IntegerType())) \
                      .withColumn("no_of_losses", col("matches_played") - col("no_of_wins" ))
final_df.show()

+---------+--------------+
|team_name|matches_played|
+---------+--------------+
|       SL|             2|
|    India|             2|
|      Eng|             2|
|       SA|             1|
|      Aus|             2|
|       NZ|             1|
+---------+--------------+

+---------+--------------+------+
|team_name|matches_played|Winner|
+---------+--------------+------+
|       SL|             2|  NULL|
|    India|             2| India|
|    India|             2| India|
|      Eng|             2|   Eng|
|       SA|             1|  NULL|
|      Aus|             2|   Aus|
|       NZ|             1|    NZ|
+---------+--------------+------+

+---------+--------------+----------+
|team_name|matches_played|no_of_wins|
+---------+--------------+----------+
|       SL|             2|         0|
|    India|             2|         2|
|      Eng|             2|         1|
|       SA|             1|         0|
|      Aus|             2|         1|
|       NZ|             1|         1|
+---------+-

In [8]:
data = [
    (1, 1, "N"),
    (2, 1, "Y"),
    (2, 2, "N"),
    (3, 3, "N"),
    (4, 2, "N"),
    (4, 3, "Y"),
    (4, 4, "N")
]
columns = ["employee_id", "department_id", "primary_flag"]

df = spark.createDataFrame(data, columns)
df.show()

+-----------+-------------+------------+
|employee_id|department_id|primary_flag|
+-----------+-------------+------------+
|          1|            1|           N|
|          2|            1|           Y|
|          2|            2|           N|
|          3|            3|           N|
|          4|            2|           N|
|          4|            3|           Y|
|          4|            4|           N|
+-----------+-------------+------------+



In [9]:
import pyspark.sql.functions  as F
data =[('RCB',), ("CSK",), ("MI",), ("PBKS",)]
df = spark.createDataFrame(data, ['team'])

df.show()

df1= df.alias("t1").crossJoin(df.alias("t2")) \
        .where(F.col("t1.team") != F.col("t2.team")) \
        .select(F.col("t1.team").alias("team1"), F.col("t2.team").alias("team2"))

df1.show()

df2= df1.filter(F.col("team1") > F.col("team2"))
df2.show()

+----+
|team|
+----+
| RCB|
| CSK|
|  MI|
|PBKS|
+----+

+-----+-----+
|team1|team2|
+-----+-----+
|  RCB|  CSK|
|  CSK|  RCB|
|  RCB|   MI|
|  RCB| PBKS|
|  CSK|   MI|
|  CSK| PBKS|
|   MI|  RCB|
|   MI|  CSK|
| PBKS|  RCB|
| PBKS|  CSK|
|   MI| PBKS|
| PBKS|   MI|
+-----+-----+

+-----+-----+
|team1|team2|
+-----+-----+
|  RCB|  CSK|
|  RCB|   MI|
|  RCB| PBKS|
|   MI|  CSK|
| PBKS|  CSK|
| PBKS|   MI|
+-----+-----+



In [ ]:
###### ASSIGNMENT